In [1]:
###Script for datamining a given CFRadial file for the necessary predictors to feed to JMLQC model 

###Need to apply controls here to ensure that dimensionality stays the same throughout the processing
###Even if we do end up flattening it to put into h5 

###May need to think about refactoring this in the case that there is a variety of functions 

###Iterate through and see what all the data types are for the parameters we are going to calculate?
###This will help for optimization purposes

###Data uses the "tidy" format - each gate is represented by a (1) x (n_features) vector - a row in the "X" dataset
###As such, the Y set will be represented as a (n_gates) x (1) matrix (Could also be described as a column vector


using NCDatasets
using HDF5
using ArgParse 
using BenchmarkTools 

In [24]:
include("./JMLQC_utils.jl")
using .JMLQC_utils

In [50]:
###Prefix of functions for calculating spatially averaged variables in the utils.jl file 
func_prefix = "calc_"
func_regex = r"(\w{1,})\((\w{1,})\)"
###Currently defined functions 
valid_funcs = ["AVG", "ISO", "STD"]

###Int representation of MISSING values 
FILL_VAL = -32000.

-32000.0

In [ ]:
###Overall structure 
###Give script desired variables, input file (or list thereof?), and output file 
###script will compute desired variables from the data, and output into and h5 file? 

In [350]:
###Define queues for function queues

function parse_commandline()
    
    s = ArgParseSettings()

    @add_arg_table s begin
        
        "CFRad_path"
            help = "Path to input CFRadial File"
            required = true
        "--argfile","-f"
            help = ("File containing comma-delimited list of variables you wish the script to calculate and output\n
                    Currently supported funcs include AVG(var_name), STD(var_name), and ISO(var_name)\n
                    Example file content: DBZ, VEL, AVG(DBZ), AVG(VEL), ISO(DBZ), STD(DBZ)\n")
        "--outfile", "-o"
            help = "Location to output mined data to"
            default = "./mined_data.h5"
        "--mode", "-m" 
            help = ("FILE or DIRECTORY\nDescribes whether the CFRAD_path describes a file or a director\n
                    DEFAULT: FILE")
            default = "FILE"
        "--QC", "-Q"
            help = ("TRUE or FALSE\nDescribes whether the given file or files contain manually QCed fields\n
                    DEFAULT: FALSE")
    end

    return parse_args(s)
end

function main()
    
    parsed_args = parse_commandline()
#     println("Parsed args:")
#     for (arg,val) in parsed_args
#         println("  $arg  =>  $val")
#     end
    ##Load given netCDF file 
    
    cfrad = NCDataset(parsed_args["CFRad_path"])
    cfrad_dims = (cfrad.dim["range"], cfrad.dim["time"])
    println("\nDIMENSIONS: $cfrad_dims\n")

    valid_vars = keys(cfrad)
    
    tasks = get_task_params(parsed_args["argfile"], valid_vars)
    
    fid = h5open(parsed_args["outfile"], "w")
    attributes(fid)["Parameters"] = tasks
    attributes(fid)["MISSING_FILL_VALUE"] = FILL_VAL
    ###Features array 
    X = Matrix{Union{Float64,Missing}}(undef,cfrad.dim["time"] * cfrad.dim["range"], length(tasks))
    
    for (i, task) in enumerate(tasks)
  
        ###λ identifier indicates that the requested task is a function 
        if (task[1] == 'λ')
            
            curr_func = lowercase(task[3:5])
            var = task[7:9]
            
            println("CALCULATING $curr_func OF $var... ")
            
            curr_func = Symbol(func_prefix * curr_func)
            startTime = time() 

            raw = @eval $curr_func($cfrad[$var][:,:])[:]
            filled = Vector{Float64}
            filled = [ismissing(x) ? Float64(FILL_VAL) : Float64(x) for x in raw]
            X[:, i] = filled[:]
            calc_length = time() - startTime
            println("Completed in $calc_length s"...)
            println()
            
        else 
            println("GETTING: $task...")
            X[:,i] = [Float64(x) for x in cfrad[task][:]]
        end 
    end 
    write_dataset(fid, "X", X)
    close(fid)
end

main()

Parsed args:
  CFRad_path  =>  /Users/ischluesche/Library/Jupyter/runtime/kernel-25c0bae8-1a5b-43f0-9f68-82adf3446daa.json
  QC  =>  nothing
  outfile  =>  ./mined_data.h5
  argfile  =>  nothing
  mode  =>  FILE
Dict{String, Any}("CFRad_path" => "/Users/ischluesche/Library/Jupyter/runtime/kernel-25c0bae8-1a5b-43f0-9f68-82adf3446daa.json", "QC" => nothing, "outfile" => "./mined_data.h5", "argfile" => nothing, "mode" => "FILE")

LoadError: NetCDF error: [31mOpening path /Users/ischluesche/Library/Jupyter/runtime/kernel-25c0bae8-1a5b-43f0-9f68-82adf3446daa.json: NetCDF: Unknown file format[39m (NetCDF error code: -51)

In [14]:

###Parses given parameter file and ensures that specified variables are found within the 
###passed CFradial file
###Could potentially internally return this as queues for each function 

##TODO case insensitivity 
function get_task_params(params_file, variablelist; delimiter=",")
    
    tasks = readlines(params_file)
    task_param_list = String[]
    
    for line in tasks
        if line[1] == "#"
            continue
        else
            delimited = split(line, delimiter)
            for token in delimited
                token = strip(token, ' ')
                expr_ret = match(func_regex,token)
                if (typeof(expr_ret) != Nothing)
                    if (expr_ret[1] ∉ valid_funcs || expr_ret[2] ∉ variablelist)
                        print(("ERROR: CANNOT CALCULATE $(expr_ret[1]) of $(expr_ret[2])\nPotentially invalid function or missing variable"))
                    else
                        print("CACLULATE $(expr_ret[1]) of $(expr_ret[2])\n")
                        ###Add λ to the front of the token to indicate it is a function call
                        push!(task_param_list, "λ" * token)
                    end 
                else
                    if token in variablelist
                        push!(task_param_list, token)
                    else
                        print("\"$token\" NOT FOUND IN CFRAD FILE.... CONTINUING...\n")
                    end
                end
            end
        end 
    end 
    
    return(task_param_list)
end 

get_task_params (generic function with 1 method)

In [12]:
@benchmark strip(" TESTING", ' ')

BenchmarkTools.Trial: 10000 samples with 988 evaluations.
 Range (min … max):  48.245 ns … 138.621 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     48.414 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   49.081 ns ±   3.345 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▅    ▂▁   ▁▂▁                                               ▁
  █████████▇█████▆▅▅▅▅▄▅▄▄▄▃▅▅▅▅▅▅▅▄▅▄▄▄▃▃▄▃▄▃▃▄▃▄▃▃▅▄▄▃▄▄▃▂▃▂ █
  48.2 ns       Histogram: log(frequency) by time        63 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [4]:
currset = Dataset("CFRADIAL/cfrad.19950516_221944.169_to_19950516_221946.124_TF-ELDR_AIR.nc")

Dataset: CFRADIAL/cfrad.19950516_221944.169_to_19950516_221946.124_TF-ELDR_AIR.nc
Group: /

Dimensions
   time = 182
   range = 384
   sweep = 1
   string_length_8 = 8
   string_length_32 = 32
   status_xml_length = 1
   r_calib = 1
   frequency = 4

Variables
  volume_number  
    Attributes:
     long_name            = data_volume_index_number
     units                = 
     _FillValue           = -9999

  platform_type   (32)
    Datatype:    Char (Char)
    Dimensions:  string_length_32
    Attributes:
     long_name            = platform_type
     options              = fixed, vehicle, ship, aircraft_fore, aircraft_aft, aircraft_tail, aircraft_belly, aircraft_roof, aircraft_nose, satellite_orbit, satellite_geostat

  primary_axis   (32)
    Datatype:    Char (Char)
    Dimensions:  string_length_32
    Attributes:
     long_name            = primary_axis_of_rotation
     options              = axis_z, axis_y, axis_x, axis_z_prime, axis_y_prime, axis_x_prime

  status_xml   (1)
 

In [8]:
x = Array{Union{Float32, Missing}}(empty,currset.dim["time"] * currset.dim["range"],2)

LoadError: MethodError: no method matching (Array{Union{Missing, Float32}})(::typeof(empty), ::Int64, ::Int64)

[0mClosest candidates are:
[0m  (Array{T})([91m::LinearAlgebra.UniformScaling[39m, ::Integer, ::Integer) where T
[0m[90m   @[39m [36mLinearAlgebra[39m [90m/Applications/Julia-1.9.app/Contents/Resources/julia/share/julia/stdlib/v1.9/LinearAlgebra/src/[39m[90m[4muniformscaling.jl:512[24m[39m
[0m  (Array{T})([91m::Nothing[39m, ::Any...) where T
[0m[90m   @[39m [90mBase[39m [90m[4mbaseext.jl:45[24m[39m
[0m  (Array{T})([91m::UndefInitializer[39m, ::Int64, ::Int64) where T
[0m[90m   @[39m [90mCore[39m [90m[4mboot.jl:492[24m[39m
[0m  ...


In [17]:
toy_valid_vars = ["DBZ"]
tasks = get_task_params("script_args.txt", toy_valid_vars )

CACLULATE AVG of DBZ
CACLULATE ISO of DBZ


2-element Vector{String}:
 "λAVG(DBZ)"
 "λISO(DBZ)"

In [405]:
for (i, task) in enumerate(tasks)
    println(i)
    println(task)
end

1
DBZ
2
λAVG(DBZ)


In [67]:
@benchmark result = [ismissing(x) ? FILL_VAL : x for x in cfrad["DBZ"][:]]

BenchmarkTools.Trial: 3635 samples with 1 evaluation.
 Range (min … max):  1.081 ms …  20.403 ms  ┊ GC (min … max):  0.00% … 93.66%
 Time  (median):     1.191 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   1.372 ms ± 929.995 μs  ┊ GC (mean ± σ):  10.33% ± 13.45%

  ▆█▆▄▂▁                                                      ▁
  ██████▇▆▅▅▅▄▅▆▄▄▁▃▃▅▃▆▃▆▅▅▅▅▄▄▆▅▄▅▅▄▁▆▃▄▃▃▄▄▅▁▄▄▅▄▄▄▄▅▃▁▅▄▄ █
  1.08 ms      Histogram: log(frequency) by time      5.68 ms <

 Memory estimate: 1.58 MiB, allocs estimate: 6005.

In [65]:
test = cfrad["DBZ"][:][map(x -> ismissing(x), cfrad["DBZ"][:])] .= 3.2

65946-element view(::Vector{Union{Missing, Float32}}, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11  …  69879, 69880, 69881, 69882, 69883, 69884, 69885, 69886, 69887, 69888]) with eltype Union{Missing, Float32}:
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 ⋮
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 3.2f0
 3.2f0

In [377]:
tasks = get_t

LoadError: MethodError: no method matching append!(::Type{Array}, ::Vector{Union{Missing, Float32}})

[0mClosest candidates are:
[0m  append!([91m::StatsBase.AbstractHistogram{T, 1}[39m, ::AbstractVector) where T
[0m[90m   @[39m [35mStatsBase[39m [90m~/.julia/packages/StatsBase/XgjIN/src/[39m[90m[4mhist.jl:295[24m[39m
[0m  append!([91m::StatsBase.AbstractHistogram{T, 1}[39m, ::AbstractVector, [91m::AbstractVector[39m) where T
[0m[90m   @[39m [35mStatsBase[39m [90m~/.julia/packages/StatsBase/XgjIN/src/[39m[90m[4mhist.jl:296[24m[39m
[0m  append!([91m::DataStructures.CircularBuffer[39m, ::AbstractVector)
[0m[90m   @[39m [33mDataStructures[39m [90m~/.julia/packages/DataStructures/MKv4P/src/[39m[90m[4mcircular_buffer.jl:128[24m[39m
[0m  ...


In [267]:
starttime = time()
tasks = get_task_params("./test_mining_config.txt", keys(currset))
timing = time() - starttime

"VEL" NOT FOUND IN CFRAD FILE.... CONTINUING...
CACLULATE AVG of DBZ
"second line" NOT FOUND IN CFRAD FILE.... CONTINUING...


0.0625758171081543

In [10]:
tasks = ["λAVG(DBZ)"]

for task in tasks
    println("GETTING: $task...")
    ###λ identifier indicates that the requested task is a function 
    print(task)
    if (task[1] == 'λ')
        print("λ")
        println(task)
        curr_func = lowercase(task[3:5])
        var = task[7:9]
        println("CALCULATING $curr_func of $var")
        curr_func = Symbol(func_prefix * curr_func)
        print( @eval $curr_func(currset[$var][:,:]))
    end 
end 

GETTING: λAVG(DBZ)...
λAVG(DBZ)λλAVG(DBZ)
CALCULATING avg of DBZ


[-34.98999786376953 -34.98999786376953 -34.98999786376953 -34.98999786376953 -34.98999786376953 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN -34.98999786376953 -34.98999786376953 NaN -34.98999786376953 -34.98999786376953 -23.783331910769146 -17.74249893426895 -13.51142770903451 -12.205999219417572 -9.358332682400942 -7.599999408309277 -6.449999450572899 -6.440713702567986 -6.315713702567986 -5.762856560094016 -5.772142265524183 -5.780713685921261 -5.736427979809897 -5.182856508663723 -5.557142223630633 -5.396427937916347 -4.941427937575749 -4.673570786203657 -5.075713668550764 -5.263570836612156 -5.361427988324847 -5.8792851482118875 -6.119999442781721 -5.843570862497602 -

 11.665000061194101 11.633750061194101 11.924999992052713 12.221666594346365 12.758333186308542 13.482083141803741 14.112916390101114 14.279583056767782 14.055416405200958 13.399166504542032 12.976841976768092 12.377857037952968; NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN 18.125 19.591428211757115 19.783332983652752 19.150624632835388 18.86571384611584 18.753332773844402 18.1899995803833 17.3462495803833 16.8204163312912 16.247083107630413 15.382083257039389 14.694583296775818 14.220833341280619 13.25166666507721 12.762083371480307 12.382083376248678 11.809583385785421 11.340833385785421 11.2

 21.19458321730296 21.72041654586792 21.804166475931805 21.996666471163433 22.298749804496765 22.668333133061726 22.742082993189495 22.846249620119732 23.29458288351695 23.56541617711385 23.643332878748577 23.94499961535136 24.043749650319416 24.059166351954143 23.507083018620808 22.455416282018025 21.20041624704997 20.023749510447185 19.285788987812243 18.601428031921387; NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN 21.9399995803833 16.59285695212228 15.28499992688497 14.643749952316284 14.324761844816662 13.622916658719381 13.132916688919067 12.684583306312561 12.44041665395101 12.247

 27.586249510447185 28.294582843780518 28.477082808812458 27.951249440511067 27.299582878748577 26.84124954541524 26.19541621208191 25.232082843780518 24.487082878748577 23.69041617711385 22.80999954541524 22.3462495803833 21.85166621208191 21.377499620119732 21.247082988421123 21.210832953453064 20.585416356722515 19.517083168029785 18.194583117961884 16.788333137830097 14.340416491031647 12.38791654507319 12.060416479905447 11.673684069984837 10.574285677501134; NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN 14.5 13.422500014305115 12.865833361943563 13.192352968103746 13.3368182182

...

 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN; NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN Na

 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN; NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN Na

 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN; NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN; NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN N

 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN; NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN Na

 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN; NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN Na

In [9]:
curr_fun

LoadError: UndefVarError: `curr_func` not defined

In [6]:
@benchmark calc_avg(currset["DBZ"][:,:])

BenchmarkTools.Trial: 125 samples with 1 evaluation.
 Range (min … max):  38.561 ms …  42.063 ms  ┊ GC (min … max): 5.20% … 6.43%
 Time  (median):     40.621 ms               ┊ GC (median):    9.65%
 Time  (mean ± σ):   40.044 ms ± 987.631 μs  ┊ GC (mean ± σ):  8.08% ± 2.19%

  ▂  ▆▄                                ▆██▆ ▂                   
  █▆███▅▃▃▃▁▁▃▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▃▇▇██████▃▄▃▁▁▁▁▁▁▁▁▁▁▁▁▃▃ ▃
  38.6 ms         Histogram: frequency by time         41.9 ms <

 Memory estimate: 71.40 MiB, allocs estimate: 840737.

In [428]:
ret

In [419]:

curr_func = Symbol("calc_" * "avg")
@eval $curr_func


calc_avg (generic function with 1 method)

In [301]:
field = "DBZ"
to_get = Symbol("currset")

:currset

In [36]:

cfrad = NCDataset("CFRADIAL/cfrad.19950516_221944.169_to_19950516_221946.124_TF-ELDR_AIR.nc")
var = "DBZ"
curr_func = "avg"
curr_func = Symbol(func_prefix * curr_func)
@eval $curr_func(cfrad[$var][:,:])[:]

69888-element Vector{Float64}:
 -34.98999786376953
 -34.98999786376953
 -34.98999786376953
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
   ⋮
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN

In [38]:
X = Matrix{Union{Float64,Missing}}(undef,cfrad.dim["time"] * cfrad.dim["range"], length(tasks))
for (i, task) in enumerate(tasks)
  
        ###λ identifier indicates that the requested task is a function 
        if (task[1] == 'λ')
            
            curr_func = lowercase(task[3:5])
            var = task[7:9]
            
            println("CALCULATING $curr_func OF $var")
            
            curr_func = Symbol(func_prefix * curr_func)
            X[:, i] = (@eval $curr_func(cfrad[$var][:,:]))[:]
            
        else 
            println("GETTING: $task...")
            X[:,i] = cfrad[task][:]
        end 
end 

CALCULATING avg OF DBZ
CALCULATING iso OF DBZ


In [49]:
create_dataset(fid, "/X", Float64, (384,182))

🔢 HDF5.Dataset: /X (file: test.h5 xfer_mode: 0)

In [74]:
DBZ_raw = cfrad["DBZ"][:]
DBZ_filled = Vector{Float64}
DBZ_filled = [ismissing(x) ? Float64(FILL_VAL) : Float64(x) for x in DBZ_raw]

69888-element Vector{Float64}:
    -34.98999786376953
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
      ⋮
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0

In [73]:
Float64(3)

3.0

In [77]:
rm("test.h5")

In [78]:
fid = h5open("test.h5", "w")
write_dataset(fid, "X", DBZ_filled)
attributes(fid["X"])["Column Names"] = ["AVG(DBZ), DBZ, VV"]
close(fid)

In [79]:
currfile = h5open("test.h5", "r")

🗂️ HDF5.File: (read-only) test.h5
└─ 🔢 X
   └─ 🏷️ Column Names

In [80]:
currfile["X"][:]

69888-element Vector{Float64}:
    -34.98999786376953
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
      ⋮
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0
 -32000.0

In [28]:
write_dataset(fid, "X", [2,3,4])

LoadError: cannot create dataset: object "X" already exists at /

In [ ]:
###Need to apply controls here to ensure that dimensionality stays the same throughout the processing
###Even if we do end up flattening it to put into h5

In [214]:
attributes(fid["X"])["test"] = "test"

"test"

In [52]:
replace!(cfrad["DBZ"][:,:], (Missing, 3))

LoadError: MethodError: no method matching replace!(::Matrix{Union{Missing, Float32}}, ::Tuple{DataType, Int64})

[0mClosest candidates are:
[0m  replace!([91m::Union{Function, Type}[39m, ::Any; count)
[0m[90m   @[39m [90mBase[39m [90m[4mset.jl:648[24m[39m
[0m  replace!(::Any, [91m::Pair...[39m; count)
[0m[90m   @[39m [90mBase[39m [90m[4mset.jl:606[24m[39m


In [366]:
x[1]= currset["DBZ"][:]

LoadError: MethodError: [0mCannot `convert` an object of type [92mVector{Union{Missing, Float32}}[39m[0m to an object of type [91mFloat32[39m

[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::Base.TwicePrecision[39m) where T<:Number
[0m[90m   @[39m [90mBase[39m [90m[4mtwiceprecision.jl:273[24m[39m
[0m  convert(::Type{T}, [91m::AbstractChar[39m) where T<:Number
[0m[90m   @[39m [90mBase[39m [90m[4mchar.jl:185[24m[39m
[0m  convert(::Type{T}, [91m::CartesianIndex{1}[39m) where T<:Number
[0m[90m   @[39m [90mBase[39m [90m[4mmultidimensional.jl:127[24m[39m
[0m  ...


In [245]:
attributes(fid["X"])["Column Names"] = ["TEST"]

LoadError: HDF5.API.H5Error: Error creating attribute Column Names for object /X
libhdf5 Stacktrace:
 [1] [0m[1mH5A__create[22m: Attribute/Object already exists
[90m     attribute already exists[39m
  ⋮

In [243]:
fid["X"]["Column Names"][]

1-element Vector{String}:
 "AVG(DBZ), DBZ, VV"

LoadError: UndefVarError: `modify` not defined

In [274]:
test_verify = h5open("./VERIFY_PROCESSING/Isaac_test.h5")

🗂️ HDF5.File: (read-only) ./VERIFY_PROCESSING/Isaac_test.h5
├─ 🔢 X
└─ 🔢 Y

In [279]:
attributes(test_verify["X"])

🔢 Attributes of HDF5.Dataset: /X (file: ./VERIFY_PROCESSING/Isaac_test.h5 xfer_mode: 0)